In [219]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import statsmodels.formula.api as smf

In [249]:
data_to_train_full = pd.read_csv("../../2.Export/2.Final/data_to_train_1.csv")
data_to_train_full = data_to_train_full.drop(['Unnamed: 0','log_precio','id_inmueble','SCaCodigo','LocCodigo','descripcion','log_area'],axis='columns')
data_to_train_full.columns

Index(['area', 'edad', 'estrato', 'extras', 'latitude', 'longitude',
       'num_banos', 'num_garages', 'numero_habitaciones', 'piso_interior',
       'piso_ubicacion', 'precio', 'precio_administracion', 'tipo_inmueble',
       'sector_catastral', 'SCaTipo', 'CountHospedaje', 'CountBibliotecas',
       'CountCanchasSinteticas', 'CountCicloParqueadero', 'CountColegios',
       'CountConsultorios', 'CountEPS', 'CountFarmacia', 'CountBomberos',
       'CountIPS', 'CountIPSAtencionInfantil', 'CountVacunacionInfantil',
       'CountMuseos', 'CountSITP', 'CountParquesUrbanos',
       'CountUnidadServicioSalud', 'CountTeatros', 'LocNombre',
       'CountRestaurantes', 'CountIglesias', 'CountCAIS',
       'CountGastronomiaBar', 'CountDelitosSexuales', 'CountHurtos', 'zona'],
      dtype='object')

In [212]:
cols = ['edad','extras','tipo_inmueble','sector_catastral','SCaTipo','LocNombre','zona']
for c in cols:
    lbl = LabelEncoder() 
    lbl.fit(list(data_to_train_full[c].values)) 
    data_to_train_full[c] = lbl.transform(list(data_to_train_full[c].values))

In [250]:
data_to_train = data_to_train_full.copy()
# cols_drop = ['CountHospedaje', 'CountBibliotecas',
#        'CountCanchasSinteticas', 'CountCicloParqueadero', 'CountColegios',
#        'CountConsultorios', 'CountEPS', 'CountFarmacia', 'CountGastronomiaBar',
#        'CountBomberos', 'CountIPS', 'CountIPSAtencionInfantil',
#        'CountVacunacionInfantil', 'CountMuseos', 'CountSITP',
#        'CountParquesUrbanos', 'CountUnidadServicioSalud', 'CountTeatros',
#              'CountRestaurantes', 'CountDelitosSexuales', 'CountHurtos',
#        'CountIglesias', 'CountCAIS',
#              'SCaTipo','LocNombre','extras','precio_administracion','area','latitude', 'longitude',
#             'sector_catastral','zona','tipo_inmueble','numero_habitaciones',
#             'edad', 'num_garages','num_banos','piso_interior', 'piso_ubicacion' ]
# # cols_drop = ['num_garages']
# data_to_train = data_to_train.drop(cols_drop,axis='columns')

#data_to_train.columns

In [196]:
model1 = linear_model.LinearRegression()
y = data_to_train['precio']
del data_to_train['precio']
X = data_to_train.values
y = y.values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=7)
model1.fit(X_train, y_train)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [204]:
print("Predict value " + str((model1.predict([X_test[142]]))))
print("Real value " + str((y_test[142])))
print("Accuracy --> ", model1.score(X_test, y_test)*100)

Predict value [8.02889513e+08]
Real value 1050000000.0
Accuracy -->  54.8363397089235


In [217]:
model2 = RandomForestRegressor(n_estimators=100)
model2.fit(X_train, y_train)
print("Accuracy --> ", model2.score(X_test, y_test)*100)


Accuracy -->  90.39937247566382


In [218]:
print("Predict value " + str(float(model2.predict([X_test[142]]))))
print("Real value " + str((y_test[142])))

Predict value 1196433000.0
Real value 1050000000.0


In [251]:
np.random.seed(2019) # a seed makes the analysis reproducible
                     # so everyone will get the same results
ndata = len(data_to_train)
# Randomly choose 0.8n indices between 1 and n
idx_train = np.random.choice(range(ndata),int(0.8*ndata),replace=False)
# The test set is comprised from all the indices that were
# not selected in the training set:
idx_test  = np.asarray(list(set(range(ndata)) - set(idx_train)))
train     = data_to_train.iloc[idx_train] # the training data set
test      = data_to_train.iloc[idx_test]  # the test data set
print(train.shape) # 19,312 rows and 19 columns
print(test.shape)  #  4,829 rows and 19 columns

(71176, 41)
(17795, 41)


In [256]:
model_log = smf.ols(formula = "precio ~ tipo_inmueble+numero_habitaciones+edad+num_garages"
                    "+num_banos+piso_interior+piso_ubicacion+precio_administracion+zona+estrato+LocNombre",
                 data = train).fit()
model_log.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 precio   R-squared:                       0.524
Model:                            OLS   Adj. R-squared:                  0.524
Method:                 Least Squares   F-statistic:                     1398.
Date:                Sat, 07 Dec 2019   Prob (F-statistic):               0.00
Time:                        18:43:11   Log-Likelihood:            -1.5398e+06
No. Observations:               71174   AIC:                         3.080e+06
Df Residuals:                   71117   BIC:                         3.080e+06
Df Model:                          56                                         
Covariance Type:            nonrobust                                         
===================================================================================================
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
Intercept                       -4.575e+08   6.05e+08     -0.757      0.449   -1.64e+09    7.28e+08
tipo_inmueble[T.Apartamento]     8.153e+07   2.15e+07      3.789      0.000    3.94e+07    1.24e+08
tipo_inmueble[T.Casa]            2.131e+08   2.29e+07      9.320      0.000    1.68e+08    2.58e+08
tipo_inmueble[T.Casa Campestre]  2.225e+08   8.04e+07      2.768      0.006    6.49e+07     3.8e+08
tipo_inmueble[T.Casa Lote]       8.001e+08   4.14e+07     19.311      0.000    7.19e+08    8.81e+08
tipo_inmueble[T.Proyecto Nuevo] -1.953e+08    6.8e+07     -2.872      0.004   -3.29e+08    -6.2e+07
edad[T.1 a 8 años]               -2.66e+08   6.03e+08     -0.441      0.659   -1.45e+09    9.16e+08
edad[T.16 a 30 años]            -3.397e+08   6.03e+08     -0.563      0.573   -1.52e+09    8.42e+08
edad[T.2]                        1.664e+07   8.52e+08      0.020      0.984   -1.65e+09    1.69e+09
edad[T.5]                       -4.061e+08   8.52e+08     -0.477      0.634   -2.08e+09    1.26e+09
edad[T.9 a 15 años]             -2.937e+08   6.03e+08     -0.487      0.626   -1.48e+09    8.88e+08
edad[T.Entre 0 y 5 años]        -1.332e+08   6.02e+08     -0.221      0.825   -1.31e+09    1.05e+09
edad[T.Entre 10 y 20 años]      -2.326e+08   6.02e+08     -0.386      0.699   -1.41e+09    9.48e+08
edad[T.Entre 5 y 10 años]       -2.169e+08   6.02e+08     -0.360      0.719    -1.4e+09    9.64e+08
edad[T.Menos de 1 año]          -1.834e+08   6.03e+08     -0.304      0.761   -1.37e+09    9.99e+08
edad[T.Más de 20 años]          -3.247e+08   6.02e+08     -0.539      0.590   -1.51e+09    8.56e+08
edad[T.Más de 30 años]          -2.851e+08   6.03e+08     -0.473      0.636   -1.47e+09    8.97e+08
edad[T.None]                    -1.797e+08   6.03e+08     -0.298      0.766   -1.36e+09       1e+09
edad[T.Remodelado]               -1.88e+08   6.03e+08     -0.312      0.755   -1.37e+09    9.94e+08
edad[T.ntre 0 y 5 años]         -4.329e+08   6.29e+08     -0.688      0.491   -1.67e+09       8e+08
zona[T.Chapinero]                -1.04e+08   3.63e+07     -2.864      0.004   -1.75e+08   -3.28e+07
zona[T.Guaymaral]                1.722e+08   7.05e+07      2.443      0.015     3.4e+07     3.1e+08
zona[T.Noroccidente]            -1.388e+08   3.55e+07     -3.913      0.000   -2.08e+08   -6.93e+07
zona[T.Norte]                    8.988e+07   3.43e+07      2.618      0.009    2.26e+07    1.57e+08
zona[T.Occidental]              -1.772e+08    3.6e+07     -4.927      0.000   -2.48e+08   -1.07e+08
zona[T.Sur]                     -5.076e+07   4.25e+07     -1.195      0.232   -1.34e+08    3.25e+07
zona[T.Zona Centro]              4.218e+07   4.33e+07      0.974      0.330   -4.27e+07    1.27e+08
zona[T.Zona Chapinero]          -1.911e+06   4.39e+07     -0.043      0.965    -8.8

In [257]:
np.round(
    np.mean(                                                      # Mean
        np.abs(                                                   # Absolute
            model_log.predict(test) - test.precio    # Error
        )
    ),
2)

341837895.16

In [258]:
np.round(
    np.sqrt(                                                          # Root
        np.mean(                                                      # Mean
            np.square(                                                # Squared
                model_log.predict(test) - test.precio     # Error
            )
        )
    ),
    decimals=3 # round to 3 decimals
)

594726018.811

In [259]:
model_log.aic

3079735.270475641